# Grid Search

Calling all models, far and wide!  Now is the time to battle to the DEATH!!!!

Just kidding, this is just a 

In [31]:
# notebooks use their location as their working directory, so
# if we are in a subfolder, move to the main folder.  
# This however can safely be run multiple times
#setwd(M:/lecospec/lecospec)
if(!dir.exists("Functions/")){
    setwd("../")
    if(!dir.exists("Functions")){
        setwd("M:/lecospec/lecospec/")
    }
}
source("Functions/lecospectR.R", echo = FALSE)
# some features of this notebook will use this in the future, maybe.  I hope so, at least.
library(IRdisplay)

## Load the Data file names

This loads the data file names.  These are assumed to be the same across training data, labels, test data and test labels

In [32]:
BASE_PATH <- "Data/gs/"
X_TRAIN_PATH <- paste0(BASE_PATH, "x_train/")
Y_TRAIN_PATH <- paste0(BASE_PATH, "y_train/")

X_TEST_PATH <- paste0(BASE_PATH, "x_test/")
Y_TEST_PATH <- paste0(BASE_PATH, "y_test/")

In [23]:
training_data <- list.files(X_TRAIN_PATH)
#training_labels <- sort(list.files("Data/gs/y_train/"))# sort alphabetically to keep the files in the same order

In [18]:
print(training_data)

 [1] "grd_clipped_corrected.csv" "grd_clipped_minmax.csv"   
 [3] "grd_clipped_raw.csv"       "grd_clipped_robust.csv"   
 [5] "grd_clipped_standard.csv"  "grd_dropped_minmax.csv"   
 [7] "grd_dropped_raw.csv"       "grd_dropped_robust.csv"   
 [9] "grd_dropped_standard.csv"  "grd_imputed_minmax.csv"   
[11] "grd_imputed_raw.csv"       "grd_imputed_robust.csv"   
[13] "grd_imputed_standard.csv"  "grd_raw_corrected.csv"    
[15] "grd_raw_minmax.csv"        "grd_raw_raw.csv"          
[17] "grd_raw_robust.csv"        "grd_raw_standard.csv"     
[19] "img_clipped_minmax.csv"    "img_clipped_raw.csv"      
[21] "img_clipped_robust.csv"    "img_clipped_standard.csv" 
[23] "img_dropped_minmax.csv"    "img_dropped_raw.csv"      
[25] "img_dropped_robust.csv"    "img_dropped_standard.csv" 
[27] "img_imputed_minmax.csv"    "img_imputed_raw.csv"      
[29] "img_imputed_robust.csv"    "img_imputed_standard.csv" 
[31] "img_raw_minmax.csv"        "img_raw_raw.csv"          
[33] "img_raw_robust.csv

In [35]:
load_data <- function(filepath){
    data <- list()
    train_data_path <- paste0(X_TRAIN_PATH, file)
    train_labels_path <- paste0(Y_TRAIN_PATH, file)
    test_labels_path <- paste0(Y_TEST_PATH, file)
    test_data_path <- paste0(X_TEST_PATH, file)

    # verify that all the paths are valid.  If not, return NULL
    if(
        file.exists(train_data_path) &&
        file.exists(train_labels_path) &&
        file.exists(test_data_path) &&
        file.exists(test_labels_path)
    ) {
        # files all exist, so return the result
        data$x_train <- read.csv(train_data_path)
        data$y_train <- read.csv(train_labels_path)$x %>% as.factor()
        data$x_test <- read.csv(test_data_path)
        data$y_test <- read.csv(test_labels_path)$x %>% as.factor()

        return(data)
    } else {
        # a file is missing, so return NULL
        return( NULL )
    }

}

In [ ]:
model <- ranger::ranger(
        num.trees = 1000,
        case.weights = weights_by_pft,
        classification = TRUE,
        x=df,
        y=gs_train_labels[[i]]
    )
    print(model)


    model_predictions <- predict(
        model, 
        test_df
    )$prediction %>% as.factor()
    
    test_samples <- gs_samples[[i]] %>% as.factor()
    if(!("Forb" %in% levels(test_samples))){

        levels(test_samples) <- c(levels(test_samples), "Forb")
    }

    confusion_matrix <- caret::confusionMatrix(
        model_predictions, 
        test_samples,
        mode = "everything"
    )


    
    model_metadata <- list(
        uuid = model_id,
        variables = colnames(df),
        type = "Random Forest (Ranger)",
        preprocessing = gs_preprocessing[[i]],
        saved = paste0("mle/models/gs/", model_id, ".rda"),
        results = paste0("mle/experiments/gs/", model_id, "/")
    )

    metadata_save_path <- paste0("mle/metadata/", model_id, ".json")
    json_metadata_str <- rjson::toJSON(model_metadata)
    write(json_metadata_str, file=metadata_save_path)
    
    save(model, file = paste0("mle/models/gs/", model_id, ".rda"))


In [53]:
parse_name <- function(filepath){
    name_no_extension <- stringi::stri_replace_last_fixed(filepath, ".csv", "")
    name_parts <- stringi::stri_split_fixed(name_no_extension, "_")

    variables <- list(
        source = name_parts[[1]]
    )

    if(length(name_parts) > 1){
        variables$outliers <- names_parts[[2]]
    } else {
        variables$outliers <- NULL
    }

    if(length(name_parts) > 2){
        variables$preprocessing <- name_parts[[3]]
    } else {
        variables$preprocessing <- NULL
    }

    return(variables)
}

In [54]:
add_model_to_manifest <- function(model_id, variables, logpath="./gs_manifest.csv"){
    if(!file.exists(logpath)){
        header <- paste(names(variables), "model_id\n", sep=",")
        write(header, file = logpath)
    }

    line <- paste(
        variables$source,
        variables$outliers,
        variables$preprocessing,
        model_id,
        sep=","
    )
    line <- paste0(line, "\n")

    write(line, file=logpath, append = TRUE)
}

In [55]:
log_model_results <- function(model_id, confusion_matrix, distribition, custom, logpath = "./gs.log"){
    # append performance data to the logs for later comparison
    sink(file = logpath, append = TRUE)
    print("-------------------------------------------------------")
    print("---------------------- Model Data ---------------------")
    
    print(paste0("Model Type: Ranger (Random Forest)"))
    print(paste0("Data Index: ",custom))
    print(paste0("Model UUID: ", model_id))
    print("---------------------- Confusion Matrix ---------------------")
    print(confusion_matrix)
    print("---------------------- Class Distribution ---------------------")
    print(distribition)
    print("-------------------------------------------------------")
    sink(NULL)
}

In [56]:
for(file in training_data){

    print(file)
    # load all the data
    data <- load_data(file)

    if(!is.null(data)){
        
        model <- ranger::ranger(
            num.trees = 1000,
            case.weights = targets_to_weights(data$y_train),
            classification = TRUE,
            x=data$x_train,
            y=data$y_train
        )

        #add forb if it's in the training data to avoid mismatch
        if(("Forb" %in% levels(data$y_train)) && !("Forb"  %in% levels(data$y_test))){
            levels(data$y_test) <- c(levels(data$y_test), "Forb")
            }

        print(levels(data$y_test))

        # create predictions (ranger)
        model_predictions <- predict(
            model, 
            data$x_test
        )$prediction %>% as.factor()

        # generate the confusion matrix
        confusion_matrix <- caret::confusionMatrix(
            model_predictions, 
            data$y_test,
            mode = "everything"
        )

        # generate an id to uniquely identify the model
        model_id <- uuid::UUIDgenerate()

        # append performance data to the logs for later comparison
        log_model_results(
            model_id = model_id,
            confusion_matrix = confusion_matrix,
            custom = file,
            distribition = model_predictions %>% as.factor() %>% table())

        # track what levels are associated with the UUID
        add_model_to_manifest(
            model_id = model_id,
            variables = parse_name(file)
        )

        # save the model using the model UUID
        save(model, file = paste0("mle/models/gs/", model_id, ".rda"))
    }
}

[1] "grd_clipped_corrected.csv"
[1] "grd_clipped_minmax.csv"
[1] "Abiotic"        "Graminoid"      "Lichen"         "Moss"          
[5] "ShrubDecid"     "ShrubEvergreen" "TreeBroadleaf"  "TreeConifer"   
[9] "Forb"          


Warning message in confusionMatrix.default(model_predictions, data$y_test, mode = "everything"):
“Levels are not in the same order for reference and data. Refactoring data to match.”
Warning message in stri_split_fixed(str, fixed, ...):
“NAs introduced by coercion”


[1] "grd_clipped_raw.csv"
[1] "Abiotic"        "Graminoid"      "Lichen"         "Moss"          
[5] "ShrubDecid"     "ShrubEvergreen" "TreeBroadleaf"  "TreeConifer"   
[9] "Forb"          


Warning message in confusionMatrix.default(model_predictions, data$y_test, mode = "everything"):
“Levels are not in the same order for reference and data. Refactoring data to match.”
Warning message in stri_split_fixed(str, fixed, ...):
“NAs introduced by coercion”


[1] "grd_clipped_robust.csv"
[1] "Abiotic"        "Graminoid"      "Lichen"         "Moss"          
[5] "ShrubDecid"     "ShrubEvergreen" "TreeBroadleaf"  "TreeConifer"   
[9] "Forb"          


Warning message in confusionMatrix.default(model_predictions, data$y_test, mode = "everything"):
“Levels are not in the same order for reference and data. Refactoring data to match.”
Warning message in stri_split_fixed(str, fixed, ...):
“NAs introduced by coercion”


[1] "grd_clipped_standard.csv"
[1] "Abiotic"        "Graminoid"      "Lichen"         "Moss"          
[5] "ShrubDecid"     "ShrubEvergreen" "TreeBroadleaf"  "TreeConifer"   
[9] "Forb"          


Warning message in confusionMatrix.default(model_predictions, data$y_test, mode = "everything"):
“Levels are not in the same order for reference and data. Refactoring data to match.”
Warning message in stri_split_fixed(str, fixed, ...):
“NAs introduced by coercion”


[1] "grd_dropped_minmax.csv"
[1] "Abiotic"        "Forb"           "Graminoid"      "Lichen"        
[5] "Moss"           "ShrubDecid"     "ShrubEvergreen" "TreeBroadleaf" 
[9] "TreeConifer"   


ERROR: Error in table(data, reference, dnn = dnn, ...): all arguments must have the same length
